In [1]:
curr_dir = '/home/vayzenbe/GitHub_Repos/GiNN'

import sys
sys.path.insert(1, f'{curr_dir}/Models')
import os, argparse
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image, ImageOps,  ImageFilter
from itertools import chain
import pandas as pd
import numpy as np
import cornet
import model_funcs
import matplotlib.pyplot as plt
from statistics import mean
%matplotlib inline


In [2]:
stim_dir = f"{curr_dir}/Stim/"
weights_dir = f"/lab_data/behrmannlab/vlad/ginn/model_weights"

im_cond = ['cropped_face', 'schematic']


train_cond = ['imagenet_objects', 'imagenet_oneface','mixed_imagenet_vggface', 'vggface']
test_dirs = ['facegen','monkey']
n_classes = [601,1200]


#train_cond = ['general']
#n_classes = [600]
#train_cond = ['imagenet_objects']
#model_types = ['vggface']
model_epochs = [0, 1, 5, 10, 15, 20, 25, 30]


model_type = 'classify'
layer =['out', 'aIT','pIT', 'V4', 'V2', 'V1']



transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])



In [3]:
def load_files(dirName):
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
            
    return listOfFiles    

In [9]:
im_files = load_files(f'{stim_dir}/{test_dirs[0]}')
im_count = len(im_files)

In [10]:
print(im_count)

522


In [ ]:
def model_loop(model, loader):

    im, label = next(iter(loader))

    first_batch = True
    for im, label in loader:
        out = model_funcs.extract_acts(model, im)

        if first_batch == True:
            all_out = out
            first_batch = False
        else:
            all_out = torch.cat((all_out, out), dim=0)

    model_acts = all_out.cpu().detach().numpy()
    
    return model_acts




In [ ]:

for test_cond in test_dirs:
    
    im_dir = f'{stim_dir}/{test_cond}'
    im_files = load_files(im_dir)
    im_count = len(im_files)

    #set up data loader for full sample
    dataset = LoadFrames.LoadFrames(im_dir,  transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=150, shuffle=False,num_workers = 4, pin_memory=True)

    for tcn, tc  in enumerate(train_cond):
        for ee in model_epochs:
            base_model = model_funcs.load_model(model_type, tc, ee, weights_dir, n_classes[tcn])
            
            for ll in layer:
            #load model 
                model = model_funcs.remove_layer(base_model, ll)

                output, label = model_loop(model, loader)

                df = pd.DataFrame(columns = ['model_type', 'train_type', 'epoch',
                                                     'im1', 'im2', 'similarity']) 
                for ii in range(0,len(output)):

                    for kk in range(ii+1,len(output)):

                        sim = cos(output[ii], output[kk])
                        sim =sim.cpu().numpy()

                        trial_results = [model_type, tc, ee, label[ii], label[kk],sim]
                        #print(trial_results)
                        trial_results = pd.Series(trial_results, index = df.columns)
                        df = df.append(trial_results, ignore_index=True)


                df.to_csv(f'{curr_dir}/Results/other_species/{model_type}_{tc}_{ee}_{ll}_{test_cond}.csv', sep =",", index = False)